<a href="https://colab.research.google.com/github/vutl/AIO-Exercises/blob/feature%2Fimage-depth/Module%202/Image_Depth_Estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Problem 1:** Xây dựng hàm tính disparity map của hai ảnh stereo đầu vào (ảnh bên trái (L) và
ảnh bên phải (R)) theo phương thức **pixel-wise matching**

In [ ]:
import cv2
import numpy as np

disparity_range = 16

def distance(x, y):
  return abs(x - y)

def pixel_wise_matching(left_img, right_img, disparity_range, save_result = True):
  # Read left , right images then convert to grayscale
  left = cv2.imread(left_img, 0)
  right = cv2.imread(right_img, 0)

  left = left.astype(np.float32)
  right = right.astype(np.float32)

  height, width = left.shape[:2]

  # Create blank disparity map
  depth = np.zeros((height, width), np.uint8)
  scale = 16
  max_value = 255

  for y in range(height):
    for x in range(width):
      # Find j where cost has minimum value
      disparity = 0
      cost_min = max_value

      for j in range(disparity_range):
        cost = max_value if (x - j) < 0 else distance(int(left[y, x]), int(right[y, x - j]))

        if cost < cost_min:
          cost_min = cost
          disparity = j

      # Let depth at (y, x) = j ( disparity )
      # Multiply by a scale factor for visualization purpose
      depth[y, x] = disparity * scale

  if save_result == True:
    print('Saving result ... ')
    # Save results
    cv2.imwrite(f' pixel_wise_l1 . png ', depth)
    cv2.imwrite(f' pixel_wise_l1_color . png ', cv2.applyColorMap(depth, cv2.COLORMAP_JET))

  print('Done.')

  return depth

In [ ]:
import cv2
import numpy as np

def distance_l1(x, y):
    return abs(x - y)

def distance_l2(x, y):
    return (x - y) ** 2

def pixel_wise_matching_l1(left_img, right_img, disparity_range, save_result = True):
  # Đọc ảnh và chuyển sang dạng grayscale
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape[:2]

    # Khởi tạo ma trận disparity map
    depth = np.zeros((height, width), np.uint8)
    scale = 16
    max_value = 255

    for y in range(height):
        for x in range(width):
            disparity = 0
            cost_min = max_value

            for j in range(disparity_range):
                cost = max_value if (x - j) < 0 else distance_l1(int(left[y, x]), int(right[y, x - j]))
                if cost < cost_min:
                    cost_min = cost
                    disparity = j

            depth[y, x] = disparity * scale

    if save_result:
        print('Saving result...')
        cv2.imwrite('pixel_wise_l1.png', depth)
        cv2.imwrite('pixel_wise_l1_color.png', cv2.applyColorMap(depth, cv2.COLORMAP_JET))
        print('Done.')

    return depth

def pixel_wise_matching_l2(left_img, right_img, disparity_range=16, save_result=True):
    # Đọc ảnh và chuyển sang dạng grayscale
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape[:2]

    # Khởi tạo ma trận disparity map
    depth = np.zeros((height, width), np.uint8)
    scale = 16
    max_value = 255 ** 2

    for y in range(height):
        for x in range(width):
            disparity = 0
            cost_min = max_value

            for j in range(disparity_range):
                cost = max_value if (x - j) < 0 else distance_l2(int(left[y, x]), int(right[y, x - j]))
                if cost < cost_min:
                    cost_min = cost
                    disparity = j

            depth[y, x] = disparity * scale

    if save_result:
        print('Saving result...')
        cv2.imwrite('pixel_wise_l2.png', depth)
        cv2.imwrite('pixel_wise_l2_color.png', cv2.applyColorMap(depth, cv2.COLORMAP_JET))
        print('Done.')

    return depth

In [ ]:
left_img_path = 'tsukuba/left.png'
right_img_path = 'tsukuba/right.png'
disparity_range = 16
pixel_wise_result_l1 = pixel_wise_matching_l1(
    left_img_path,
    right_img_path,
    disparity_range,
    save_result=True
)

pixel_wise_result_l2 = pixel_wise_matching_l2(
    left_img_path,
    right_img_path,
    disparity_range,
    save_result=True
)

Saving result...
Done.
Saving result...
Done.


**Problem 2:** Xây dựng hàm tính disparity map của hai ảnh stereo đầu vào (ảnh bên trái (L)
và ảnh bên phải (R)) theo phương thức **window-based matching.**

In [ ]:
import cv2
import numpy as np

def distance(x, y):
    return abs(x - y)

def window_based_matching(left_img, right_img, disparity_range, kernel_size=5, save_result=True):
    # Đọc ảnh và chuyển sang dạng grayscale
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape[:2]

    # Khởi tạo ma trận disparity map
    depth = np.zeros((height, width), np.uint8)

    kernel_half = (kernel_size - 1) // 2
    scale = 3  # Hệ số tỉ lệ để chuyển đổi giá trị disparity

    for y in range(kernel_half, height - kernel_half):
        for x in range(kernel_half, width - kernel_half):
            disparity = 0
            cost_min = float('inf')

            for j in range(disparity_range):
                total_cost = 0

                for v in range(-kernel_half, kernel_half + 1):
                    for u in range(-kernel_half, kernel_half + 1):
                        if (x + u - j) >= 0:
                            # Tính tổng chi phí (cost) trong window
                            total_cost += distance(int(left[y + v, x + u]), int(right[y + v, (x + u) - j]))

                # Kiểm tra và cập nhật disparity tối ưu
                if total_cost < cost_min:
                    cost_min = total_cost
                    disparity = j

            # Gán giá trị disparity vào depth map
            depth[y, x] = disparity * scale

    # Lưu kết quả nếu cần
    if save_result:
        print('Saving result...')
        cv2.imwrite('window_based_l1.png', depth)
        cv2.imwrite('window_based_l1_color.png', cv2.applyColorMap(depth, cv2.COLORMAP_JET))
        print('Done.')

    return depth

In [ ]:
import cv2
import numpy as np

def l1_distance(x, y):
    return abs(x - y)

def l2_distance(x, y):
    return (x - y) ** 2

def window_based_matching_l1(left_img, right_img, disparity_range, kernel_size=5, save_result=True):
    # Đọc ảnh và chuyển sang dạng grayscale
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape[:2]

    # Khởi tạo ma trận disparity map
    depth = np.zeros((height, width), np.uint8)

    kernel_half = (kernel_size - 1) // 2
    scale = 3  # Hệ số tỉ lệ để chuyển đổi giá trị disparity

    for y in range(kernel_half, height - kernel_half):
        for x in range(kernel_half, width - kernel_half):
            disparity = 0
            cost_min = float('inf')

            for j in range(disparity_range):
                total_cost = 0

                for v in range(-kernel_half, kernel_half + 1):
                    for u in range(-kernel_half, kernel_half + 1):
                        if (x + u - j) >= 0:
                            total_cost += l1_distance(int(left[y + v, x + u]), int(right[y + v, (x + u) - j]))

                if total_cost < cost_min:
                    cost_min = total_cost
                    disparity = j

            depth[y, x] = disparity * scale

    if save_result:
        print('Saving result...')
        cv2.imwrite('window_based_l1.png', depth)
        cv2.imwrite('window_based_l1_color.png', cv2.applyColorMap(depth, cv2.COLORMAP_JET))
        print('Done.')

    return depth

def window_based_matching_l2(left_img, right_img, disparity_range, kernel_size=5, save_result=True):
    # Đọc ảnh và chuyển sang dạng grayscale
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape[:2]

    # Khởi tạo ma trận disparity map
    depth = np.zeros((height, width), np.uint8)

    kernel_half = (kernel_size - 1) // 2
    scale = 3  # Hệ số tỉ lệ để chuyển đổi giá trị disparity

    for y in range(kernel_half, height - kernel_half):
        for x in range(kernel_half, width - kernel_half):
            disparity = 0
            cost_min = float('inf')

            for j in range(disparity_range):
                total_cost = 0

                for v in range(-kernel_half, kernel_half + 1):
                    for u in range(-kernel_half, kernel_half + 1):
                        if (x + u - j) >= 0:
                            total_cost += l2_distance(int(left[y + v, x + u]), int(right[y + v, (x + u) - j]))

                if total_cost < cost_min:
                    cost_min = total_cost
                    disparity = j

            depth[y, x] = disparity * scale

    if save_result:
        print('Saving result...')
        cv2.imwrite('window_based_l2.png', depth)
        cv2.imwrite('window_based_l2_color.png', cv2.applyColorMap(depth, cv2.COLORMAP_JET))
        print('Done.')

    return depth

In [ ]:
left_img_path = 'Aloe/Aloe_left_1.png'
right_img_path = 'Aloe/Aloe_right_1.png'
disparity_range = 64
kernel_size = 3

# Tính toán disparity map sử dụng L1 norm
window_based_result_l1 = window_based_matching_l1(
    left_img_path,
    right_img_path,
    disparity_range,
    kernel_size=kernel_size,
    save_result=True
)

# Tính toán disparity map sử dụng L2 norm
window_based_result_l2 = window_based_matching_l2(
    left_img_path,
    right_img_path,
    disparity_range,
    kernel_size=kernel_size,
    save_result=True
)

Saving result...
Done.
Saving result...
Done.


**Problem 3:** disparity_range = 64 và kernel_size
= 5

In [ ]:
# Hàm thực hiện window-based matching với L1 norm
def window_based_matching_l1(left_img, right_img, disparity_range, kernel_size=5, save_result=True):
    # Đọc ảnh và chuyển sang dạng grayscale
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape[:2]

    # Khởi tạo ma trận disparity map
    depth = np.zeros((height, width), np.uint8)

    kernel_half = (kernel_size - 1) // 2
    scale = 3  # Hệ số tỉ lệ để chuyển đổi giá trị disparity

    for y in range(kernel_half, height - kernel_half):
        for x in range(kernel_half, width - kernel_half):
            disparity = 0
            cost_min = float('inf')

            for j in range(disparity_range):
                total_cost = 0

                for v in range(-kernel_half, kernel_half + 1):
                    for u in range(-kernel_half, kernel_half + 1):
                        if (x + u - j) >= 0:
                            total_cost += l1_distance(int(left[y + v, x + u]), int(right[y + v, (x + u) - j]))

                if total_cost < cost_min:
                    cost_min = total_cost
                    disparity = j

            depth[y, x] = disparity * scale

    if save_result:
        print('Saving result...')
        cv2.imwrite('window_based_l1_problem3.png', depth)
        cv2.imwrite('window_based_l1_problem3_color.png', cv2.applyColorMap(depth, cv2.COLORMAP_JET))
        print('Done.')

    return depth

# Hàm thực hiện window-based matching với L2 norm
def window_based_matching_l2(left_img, right_img, disparity_range, kernel_size=5, save_result=True):
    # Đọc ảnh và chuyển sang dạng grayscale
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape[:2]

    # Khởi tạo ma trận disparity map
    depth = np.zeros((height, width), np.uint8)

    kernel_half = (kernel_size - 1) // 2
    scale = 3  # Hệ số tỉ lệ để chuyển đổi giá trị disparity

    for y in range(kernel_half, height - kernel_half):
        for x in range(kernel_half, width - kernel_half):
            disparity = 0
            cost_min = float('inf')

            for j in range(disparity_range):
                total_cost = 0

                for v in range(-kernel_half, kernel_half + 1):
                    for u in range(-kernel_half, kernel_half + 1):
                        if (x + u - j) >= 0:
                            total_cost += l2_distance(int(left[y + v, x + u]), int(right[y + v, (x + u) - j]))

                if total_cost < cost_min:
                    cost_min = total_cost
                    disparity = j

            depth[y, x] = disparity * scale

    if save_result:
        print('Saving result...')
        cv2.imwrite('window_based_l2_problem3.png', depth)
        cv2.imwrite('window_based_l2_problem3_color.png', cv2.applyColorMap(depth, cv2.COLORMAP_JET))
        print('Done.')

    return depth

# Thiết lập các tham số đầu vào cho Problem 3
left_img_path = 'Aloe/Aloe_left_1.png'
right_img_path = 'Aloe/Aloe_right_2.png'
disparity_range = 64
kernel_size = 5

# Tính toán disparity map sử dụng L1 norm cho Problem 3
window_based_result_l1 = window_based_matching_l1(
    left_img_path,
    right_img_path,
    disparity_range,
    kernel_size=kernel_size,
    save_result=True
)

# Tính toán disparity map sử dụng L2 norm cho Problem 3
window_based_result_l2 = window_based_matching_l2(
    left_img_path,
    right_img_path,
    disparity_range,
    kernel_size=kernel_size,
    save_result=True
)

**Phân tích kết quả:**
Sau khi chạy đoạn code trên, ta thu được hai bộ disparity map sử dụng L1 và L2.
Khi sử dụng ảnh Aloe_left_1.png và Aloe_right_2.png, cùng với kernel size là 5 và disparity range là 64, ta có thể nhận thấy kết quả disparity map bị nhiễu nhiều hơn so với khi sử dụng các ảnh khác hoặc với kernel size nhỏ hơn.
Điều này xảy ra do sự khác biệt giữa hai ảnh đầu vào, và khi ta mở rộng kích thước cửa sổ (kernel_size = 5), ảnh hưởng của các điểm ảnh lân cận trở nên rõ rệt hơn, có thể dẫn đến mất chi tiết hoặc tăng độ nhiễu trong disparity map.

**Problem 4:** Cosine Similarity

In [ ]:
import cv2
import numpy as np

# Hàm tính Cosine Similarity giữa hai vectors
def cosine_similarity(x, y):
    numerator = np.dot(x, y)
    denominator = np.linalg.norm(x) * np.linalg.norm(y)
    return numerator / denominator if denominator != 0 else 0

# Hàm thực hiện window-based matching sử dụng Cosine Similarity
def window_based_matching_cosine_similarity(left_img, right_img, disparity_range, kernel_size=5, save_result=True):
    # Đọc ảnh và chuyển sang dạng grayscale
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape[:2]

    # Khởi tạo ma trận disparity map
    depth = np.zeros((height, width), np.uint8)

    kernel_half = (kernel_size - 1) // 2
    scale = 3  # Hệ số tỉ lệ để chuyển đổi giá trị disparity

    for y in range(kernel_half, height - kernel_half):
        for x in range(kernel_half, width - kernel_half):
            disparity = 0
            cost_optimal = -1  # Giá trị tối ưu cho Cosine Similarity là giá trị lớn nhất

            for j in range(disparity_range):
                d = x - j
                if (d - kernel_half) >= 0:
                    # Lấy cửa sổ (window) xung quanh điểm ảnh trung tâm
                    window_left = left[y - kernel_half:y + kernel_half + 1, x - kernel_half:x + kernel_half + 1]
                    window_right = right[y - kernel_half:y + kernel_half + 1, d - kernel_half:d + kernel_half + 1]

                    # Làm phẳng các cửa sổ thành vector
                    vector_left = window_left.flatten()
                    vector_right = window_right.flatten()

                    # Tính Cosine Similarity giữa hai vectors
                    cost = cosine_similarity(vector_left, vector_right)

                    # Chọn giá trị disparity tối ưu dựa trên Cosine Similarity lớn nhất
                    if cost > cost_optimal:
                        cost_optimal = cost
                        disparity = j

            # Lưu giá trị disparity tối ưu vào depth map
            depth[y, x] = disparity * scale

    if save_result:
        print('Saving result...')
        cv2.imwrite('window_based_cosine_similarity.png', depth)
        cv2.imwrite('window_based_cosine_similarity_color.png', cv2.applyColorMap(depth, cv2.COLORMAP_JET))
        print('Done.')

    return depth

# Thiết lập các tham số đầu vào cho Problem 4
left_img_path = 'Aloe/Aloe_left_1.png'
right_img_path = 'Aloe/Aloe_right_2.png'
disparity_range = 64
kernel_size = 5

# Tính toán disparity map sử dụng Cosine Similarity cho Problem 4
window_based_result_cosine = window_based_matching_cosine_similarity(
    left_img_path,
    right_img_path,
    disparity_range,
    kernel_size=kernel_size,
    save_result=True
)